In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os
from multiprocessing import Process, Queue
import sys
import time
import random
import math


max_num = 1000
train_path = '/home/Irving/AI/Alex/Data/ILSVRC2012_img_train/'
#valid_path = '/home/Irving/AI/Alex/Data/validation/'
#label_path = '/HardDisk/Irving/ILSVRC2012/label_data/'
cores = 6   # the number of cores


In [2]:

#Imagenet图片都保存在/data目录下，里面有1000个子目录，获取这些子目录的名字
classes = os.listdir(train_path)
classes.sort() 
#构建一个字典，Key是目录名，value是类名0-999
labels_dict = {}
for i in range(len(classes)):
    labels_dict[classes[i]]=i
 
#构建训练集文件列表，里面的每个元素是路径名+图片文件名+类名
images_labels_list = []
for i in range(len(classes)):
    path = train_path+classes[i]+'/'
    images_files = os.listdir(path)
    images_files.sort()
    label = str(labels_dict[classes[i]])
    for image_file in images_files:
        images_labels_list.append(path+','+image_file+','+classes[i])
random.shuffle(images_labels_list)

In [3]:
labels_dict


{'n01440764': 0,
 'n01443537': 1,
 'n01484850': 2,
 'n01491361': 3,
 'n01494475': 4,
 'n01496331': 5,
 'n01498041': 6,
 'n01514668': 7,
 'n01514859': 8,
 'n01518878': 9,
 'n01530575': 10,
 'n01531178': 11,
 'n01532829': 12,
 'n01534433': 13,
 'n01537544': 14,
 'n01558993': 15,
 'n01560419': 16,
 'n01580077': 17,
 'n01582220': 18,
 'n01592084': 19,
 'n01601694': 20,
 'n01608432': 21,
 'n01614925': 22,
 'n01616318': 23,
 'n01622779': 24,
 'n01629819': 25,
 'n01630670': 26,
 'n01631663': 27,
 'n01632458': 28,
 'n01632777': 29,
 'n01641577': 30,
 'n01644373': 31,
 'n01644900': 32,
 'n01664065': 33,
 'n01665541': 34,
 'n01667114': 35,
 'n01667778': 36,
 'n01669191': 37,
 'n01675722': 38,
 'n01677366': 39,
 'n01682714': 40,
 'n01685808': 41,
 'n01687978': 42,
 'n01688243': 43,
 'n01689811': 44,
 'n01692333': 45,
 'n01693334': 46,
 'n01694178': 47,
 'n01695060': 48,
 'n01697457': 49,
 'n01698640': 50,
 'n01704323': 51,
 'n01728572': 52,
 'n01728920': 53,
 'n01729322': 54,
 'n01729977': 55,
 '

#读取验证集的图片对应的类名标签文件
valid_classes = []
with open('imagenet_2012_validation_synset_labels.txt','r') as f:
	  valid_classes = [line.strip() for line in f.readlines()]
#构建验证集文件列表，里面的每个元素是路径名+图片文件名+类名
valid_images_labels_list = []
valid_images_files = os.listdir(valid_path)
valid_images_files.sort()      # if the order of the image file is not right , the sort function may be help
for file_item in valid_images_files:
    number = int(file_item[15:23])-1
    valid_images_labels_list.append(valid_path+','+file_item+','+valid_classes[number])

In [4]:
#valid_images_labels_list

In [5]:
#valid_classes


In [6]:
#把图像数据和标签转换为TRRECORD的格式
def make_example(image, height, width, label, bbox, text):
    colorspace = b'RGB'
    channels = 3
    img_format = b'JPEG'
    return tf.train.Example(features=tf.train.Features(feature={
        'image' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
        'height' : tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'width' : tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'channels' : tf.train.Feature(int64_list=tf.train.Int64List(value=[channels])),
        'colorspace' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[colorspace])),
        'img_format' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_format])),
        'label' : tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
        'bbox_xmin' : tf.train.Feature(float_list=tf.train.FloatList(value=bbox[0])),
        'bbox_xmax' : tf.train.Feature(float_list=tf.train.FloatList(value=bbox[2])),
        'bbox_ymin' : tf.train.Feature(float_list=tf.train.FloatList(value=bbox[1])),
        'bbox_ymax' : tf.train.Feature(float_list=tf.train.FloatList(value=bbox[3])),
        'text' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[text]))
    }))

In [7]:
#读取bbox文件
bbox_list = {}
with open('bbox_train.csv', 'r') as bboxfile:
    records = bboxfile.readlines()
    for record in records:
        fields = record.strip().split(',')
        filename = fields[0][:-4]
        xmin = [float(x) for x in fields[1].split(';')]
        ymin = [float(x) for x in fields[2].split(';')]
        xmax = [float(x) for x in fields[3].split(';')]
        ymax = [float(x) for x in fields[4].split(';')]
        bbox_list[filename] = [xmin, ymin, xmax, ymax] 


valid_text = {}
with open('/home/Irving/AI/Alex/Data/label_valid_train.txt','r') as vtfile:
    records = vtfile.readlines()
    
    for record in records:
        fields = record.strip().split('\t')
        v = fields[0]
        t = fields[2]
        valid_text[v] = t        

In [8]:
#读取Labels的描述
labels_text = {}
with open('imagenet_metadata.txt', 'r') as metafile:
    records = metafile.readlines()
    for record in records:
        fields = record.strip().split('\t')
        label = fields[0]
        text = fields[1]
        labels_text[label] = text

In [9]:
#valid_text

In [10]:
#这个函数用来生成TFRECORD文件，第一个参数是列表，每个元素是图片文件名加类名，第二个参数是写入的目录名
#第三个参数是文件名的起始序号，第四个参数是队列名称，用于和父进程发送消息
def gen_tfrecord(trainrecords, targetfolder, startnum, queue):
    tfrecords_file_num = startnum
    file_num = 0
    total_num = len(trainrecords)
    pid = os.getpid()
    queue.put((pid, file_num))
    writer = tf.python_io.TFRecordWriter(targetfolder+"train_"+str(tfrecords_file_num)+".tfrecord")
    for record in trainrecords:
        file_num += 1
        fields = record.split(',')
        img = cv2.imread(fields[0]+fields[1])
        height, width, _ = img.shape
        img_jpg = cv2.imencode('.jpg', img)[1].tobytes()
        label = labels_dict[fields[2]]
        bbox = []
        try:
            bbox = bbox_list[fields[1][:-5]]
        except KeyError:
            bbox = [[],[],[],[]]
        text = labels_text[fields[2]]
        ex = make_example(img_jpg, height, width, label, bbox, text.encode())
        writer.write(ex.SerializeToString())
        #每写入100条记录，向父进程发送消息，报告进度
        if file_num%100==0:
            queue.put((pid, file_num))
        if file_num%max_num==0:
            writer.close()
            tfrecords_file_num += 1
            writer = tf.python_io.TFRecordWriter(targetfolder+"train_"+str(tfrecords_file_num)+".tfrecord")
    writer.close()        
    queue.put((pid, file_num))
 
#这个函数用来多进程生成TFRECORD文件，第一个参数是要处理的图片的文件名列表，第二个参数是需要用的CPU核心数
#第三个参数写入的文件目录名
def process_in_queues(fileslist, cores, targetfolder):
    total_files_num = len(fileslist)
    each_process_files_num = int(total_files_num/cores)
    files_for_process_list = []
    for i in range(cores-1):
        files_for_process_list.append(fileslist[i*each_process_files_num:(i+1)*each_process_files_num])
    files_for_process_list.append(fileslist[(cores-1)*each_process_files_num:])
    files_number_list = [len(l) for l in files_for_process_list]
    
    each_process_tffiles_num = math.ceil(each_process_files_num/max_num)
    
    queues_list = []
    processes_list = []
    for i in range(cores):
        queues_list.append(Queue())
        #queue = Queue()
        processes_list.append(Process(target=gen_tfrecord, 
                                      args=(files_for_process_list[i],targetfolder,
                                      each_process_tffiles_num*i+1,queues_list[i],)))
 
    for p in processes_list:
        Process.start(p)
 
    #父进程循环查询队列的消息，并且每0.5秒更新一次
    while(True):
        try:
            total = 0
            progress_str=''
            for i in range(cores):
                msg=queues_list[i].get()
                total += msg[1]
                progress_str+='PID'+str(msg[0])+':'+str(msg[1])+'/'+ str(files_number_list[i])+'|'
            progress_str+='\r'
            print(progress_str, end='')
            if total == total_files_num:
                for p in processes_list:
                    p.terminate()
                    p.join()
                break
            time.sleep(0.5)
        except:
            break
    return total

In [11]:
if __name__ == '__main__':
    print('Start processing train data using %i CPU cores:'%cores)
    starttime=time.time()       	  
    total_processed = process_in_queues(images_labels_list, cores, targetfolder='/home/Irving/AI/Alex/Data/train_tf/')
    endtime=time.time()
    print('\nProcess finish, total process %i images in %i seconds'%(total_processed, int(endtime-starttime)))

Start processing train data using 6 CPU cores:
PID1591:213527/213527|PID1595:213527/213527|PID1597:213527/213527|PID1604:213527/213527|PID1615:213527/213527|PID1617:213532/213532|
Process finish, total process 1281167 images in 1563 seconds


def gen_tfrecord_valid(trainrecords, targetfolder, startnum, queue):
    tfrecords_file_num = startnum
    file_num = 0
    total_num = len(trainrecords)
    pid = os.getpid()
    queue.put((pid, file_num))
    writer = tf.python_io.TFRecordWriter(targetfolder + "valid_" + str(tfrecords_file_num) + ".tfrecord")
    for record in trainrecords:
        file_num += 1
        fields = record.split(',')
        img = cv2.imread(fields[0] + fields[1])
        height, width, _ = img.shape
        img_jpg = cv2.imencode('.jpg', img)[1].tobytes()
        label = labels_dict[fields[2]]
        bbox = []
        try:
            bbox = bbox_list[fields[1][:-5]]
        except KeyError:
            bbox = [[], [], [], []]
        text = labels_text[fields[2]]
        #print("###")
        #print(fields[2])
        #print(text)
        #print("###")
        
        
        ex = make_example(img_jpg, height, width, label, bbox, text.encode())
        writer.write(ex.SerializeToString())
        # 每写入100条记录，向父进程发送消息，报告进度
        if file_num % 100 == 0:
            queue.put((pid, file_num))
        if file_num % max_num == 0:
            writer.close()
            tfrecords_file_num += 1
            writer = tf.python_io.TFRecordWriter(targetfolder + "valid_" + str(tfrecords_file_num) + ".tfrecord")
    writer.close()
    queue.put((pid, file_num))
    
    
# 这个函数用来多进程生成TFRECORD文件，第一个参数是要处理的图片的文件名列表，第二个参数是需要用的CPU核心数
# 第三个参数写入的文件目录名
def process_in_queues_valid(fileslist, cores, targetfolder):
    total_files_num = len(fileslist)
    each_process_files_num = int(total_files_num / cores)
    files_for_process_list = []
    #print(files_number_list)
    for i in range(cores - 1):
        files_for_process_list.append(fileslist[i * each_process_files_num:(i + 1) * each_process_files_num])
    files_for_process_list.append(fileslist[(cores - 1) * each_process_files_num:])
    #print(1)
    files_number_list = [len(l) for l in files_for_process_list]
    #print(2)


    each_process_tffiles_num = math.ceil(each_process_files_num / max_num)

    queues_list = []
    processes_list = []
    #print(3)
    for i in range(cores):
        queues_list.append(Queue())
        # queue = Queue()
        processes_list.append(Process(target=gen_tfrecord_valid,
                                      args=(files_for_process_list[i], targetfolder,
                                            each_process_tffiles_num * i + 1, queues_list[i],)))

    for p in processes_list:
        Process.start(p)

    # 父进程循环查询队列的消息，并且每0.5秒更新一次
    while (True):
        try:
            total = 0
            progress_str = ''
            #print(4)
            for i in range(cores):
                msg = queues_list[i].get()
                total += msg[1]
                progress_str += 'PID' + str(msg[0]) + ':' + str(msg[1]) + '/' + str(files_number_list[i]) + '|'
            
            progress_str += '\r'
            print(progress_str, end='')
            if total == total_files_num:
                for p in processes_list:
                    p.terminate()
                    p.join()
                break
            time.sleep(0.5)
        except:
            break
    return total



    
if __name__ == '__main__':
    print('Start processing validation data using %i CPU cores:' % cores)
    starttime = time.time()
    total_processed = process_in_queues_valid(valid_images_labels_list, cores, targetfolder='/home/Irving/AI/Alex/Data/valid_tf/')
    endtime = time.time()
    #print('\nProcess finish, total process %i images, using %i seconds' % (total_processed, int(endtime - starttime)))
    print('\nProcess finish, total process %i images in %i seconds'%(total_processed, int(endtime-starttime)))